## 5. `VectorStoreRetrieverMemory`
- **Purpose**: Stores chat history in a vector database and retrieves relevant past utterances semantically.
- **Use‑case**: Large context windows, agent reflection, or retrieving past conversation highlights.
- **Main difference**: Retrieves only semantically relevant history based on current input :contentReference[oaicite:3]{index=3}.


In [3]:
# ================================
# Step 1: Install dependencies
# ================================
!pip install -q langchain langchain-groq sentence-transformers faiss-cpu langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [4]:
# ================================
# Step 2: Import Libraries
# ================================
from langchain_groq import ChatGroq
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain.memory import VectorStoreRetrieverMemory
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from IPython.display import Markdown, display
from google.colab import userdata

In [5]:
# ================================
# Step 3: Setup Groq LLM
# ================================
api_key = userdata.get("GROQ_API_KEY")

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    api_key=api_key,
    temperature=0.3,
)

In [24]:
from langchain_core.messages import HumanMessage
phdr="""placeholder"""

In [25]:
# ================================
# Step 4: Setup VectorStoreRetrieverMemory
# ================================

# Step 4.1: Embedding model
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Step 4.2: Initialize FAISS vector store
vectorstore = FAISS.from_texts([phdr], embedding)

memory = VectorStoreRetrieverMemory(
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    memory_key="history",  # maps to MessagesPlaceholder
)

In [26]:
# ================================
# Step 5: Prompt Template with Memory
# ================================
system_msg = SystemMessagePromptTemplate.from_template(
    "You are a helpful assistant. Recall relevant past conversations using memory."
)
human_msg = HumanMessagePromptTemplate.from_template("{input}")

chat_prompt = ChatPromptTemplate.from_messages([
    system_msg,
    MessagesPlaceholder(variable_name="history"),  # injects retrieved context
    human_msg
])

In [28]:
# ================================
# Step 6: Chat Loop with Memory
# ================================
user_inputs = [
    "Can you explain how backpropagation works in neural networks?",
    "Thanks! What about dropout in training?",
    "Remind me what you said earlier about backpropagation."
]

for input_text in user_inputs:
    # Step 1: Retrieve memory text and convert to message list
    retrieved = memory.load_memory_variables({"input": input_text})["history"]
    if retrieved.strip() == "placeholder":
        messages = []
    else:
        messages = [HumanMessage(content="Earlier you said: " + retrieved)]

    # Step 2: Format prompt with history and new input
    formatted_messages = chat_prompt.format_messages(
        input=input_text,
        history=messages
    )

    # Step 3: Get response
    response = llm.invoke(formatted_messages)

    # Step 4: Save turn into memory
    memory.save_context({"input": input_text}, {"output": response.content})

    display(Markdown(f"### User: {input_text}"))
    display(Markdown(f"** Assistant:** {response.content}"))

### User: Can you explain how backpropagation works in neural networks?

** Assistant:** **What is Backpropagation?**

Backpropagation, short for "backward propagation of errors," is an algorithm used to train artificial neural networks. It's a method for supervised learning, where the network is trained on labeled data to minimize the error between its predictions and the actual outputs.

**The Backpropagation Process**

The backpropagation process involves the following steps:

1. **Forward Pass**: The network processes the input data and produces an output. This output is compared to the actual output (label) to calculate the error.
2. **Error Calculation**: The error between the predicted output and the actual output is calculated using a loss function, such as mean squared error (MSE) or cross-entropy.
3. **Backward Pass**: The error is propagated backwards through the network, adjusting the weights and biases of each layer to minimize the error. This is done using the chain rule of calculus.
4. **Weight Update**: The weights and biases of each layer are updated based on the calculated error and the gradients of the loss function with respect to each weight and bias.
5. **Repeat**: Steps 1-4 are repeated for multiple iterations, with the network adjusting its weights and biases to minimize the error.

**Key Components of Backpropagation**

* **Activation Functions**: Each layer uses an activation function to introduce non-linearity into the network. Common activation functions include sigmoid, ReLU (rectified linear unit), and tanh.
* **Gradients**: The gradients of the loss function with respect to each weight and bias are calculated using the chain rule. These gradients are used to update the weights and biases.
* **Optimizers**: Optimizers, such as stochastic gradient descent (SGD), Adam, or RMSProp, are used to update the weights and biases based on the calculated gradients.

**Example**

Suppose we have a simple neural network with two inputs, one hidden layer with two neurons, and one output. We want to train the network to predict the output of a simple function, such as `y = 2x + 1`.

1. Forward pass: The network processes the input `x` and produces an output `y_pred`.
2. Error calculation: The error between `y_pred` and the actual output `y` is calculated using MSE.
3. Backward pass: The error is propagated backwards through the network, adjusting the weights and biases of each layer to minimize the error.
4. Weight update: The weights and biases of each layer are updated based on the calculated error and the gradients of the loss function with respect to each weight and bias.

**Conclusion**

Backpropagation is a powerful algorithm for training neural networks. By iteratively adjusting the weights and biases of each layer to minimize the error, the network can learn to make accurate predictions on complex tasks. I hope this explanation helps! Do you have any specific questions about backpropagation or neural networks?

### User: Thanks! What about dropout in training?

** Assistant:** We've already discussed dropout in training earlier. To recap, **dropout is a regularization technique used in neural networks to prevent overfitting**. It was introduced by Geoffrey Hinton and his colleagues in 2012.

**What is Dropout?**

Dropout is a simple yet effective technique that randomly sets a fraction of the neurons in a layer to zero during training. This means that during each iteration, a random subset of neurons is temporarily removed from the network, and the remaining neurons are used to make predictions.

**How Does Dropout Work?**

Here's how dropout works:

1. **Randomly Select Neurons**: During each iteration, a random subset of neurons is selected to be dropped out.
2. **Set Neurons to Zero**: The selected neurons are set to zero, effectively removing them from the network.
3. **Scale the Remaining Neurons**: The remaining neurons are scaled up to compensate for the removed neurons.
4. **Make Predictions**: The network makes predictions using the remaining neurons.
5. **Backpropagate**: The error is backpropagated through the network, adjusting the weights and biases of the remaining neurons.

**Benefits of Dropout**

Dropout has several benefits:

* **Prevents Overfitting**: By randomly removing neurons, dropout prevents the network from relying too heavily on any individual neuron or group of neurons.
* **Encourages Ensemble Learning**: Dropout encourages the network to learn multiple representations of the data, effectively creating an ensemble of models.
* **Improves Generalization**: Dropout helps the network generalize better to new, unseen data.

**Hyperparameters for Dropout**

The key hyperparameter for dropout is the dropout rate, which controls the fraction of neurons to be dropped out. A common range for the dropout rate is between 0.2 and 0.5.

**Example**

Suppose we have a neural network with a hidden layer of 100 neurons, and we want to apply a dropout rate of 0.2. During each iteration, 20 neurons (0.2 x 100) would be randomly selected to be dropped out, and the remaining 80 neurons would be used to make predictions.

**Code Example**

In Python, using the Keras library, you can implement dropout as follows:
```python
from keras.layers import Dropout

# Create a neural network model
model = Sequential()
model.add(Dense(100, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))  # Apply dropout with a rate of 0.2
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
```
In this example, the `Dropout` layer is applied to the output of the first dense layer, with a dropout rate of 0.2.

Do you have any specific questions about dropout or neural networks?

### User: Remind me what you said earlier about backpropagation.

** Assistant:** Earlier, I explained how backpropagation works in neural networks. I described it as a fundamental concept in neural networks, and an algorithm used to train artificial neural networks. 

I broke down the backpropagation process into the following steps:

1. **Forward Pass**: The network processes the input data and produces an output.
2. **Error Calculation**: The error between the predicted output and the actual output is calculated using a loss function.
3. **Backward Pass**: The error is propagated backwards through the network, adjusting the weights and biases of each layer to minimize the error.
4. **Weight Update**: The weights and biases of each layer are updated based on the calculated error and the gradients of the loss function with respect to each weight and bias.
5. **Repeat**: Steps 1-4 are repeated for multiple iterations, with the network adjusting its weights and biases to minimize the error.

I also mentioned some key components of backpropagation, including:

* **Activation Functions**: Each layer uses an activation function to introduce non-linearity into the network.
* **Gradients**: The gradients of the loss function with respect to each weight and bias are calculated using the chain rule.
* **Optimizers**: Optimizers, such as stochastic gradient descent (SGD), Adam, or RMSProp, are used to update the weights and biases based on the calculated gradients.

Additionally, I provided an example of how backpropagation works in a simple neural network, and concluded that backpropagation is a powerful algorithm for training neural networks, allowing them to learn and make accurate predictions on complex tasks. 

Would you like me to elaborate on any of these points or provide further clarification?